In [1]:
from mindspore import context
from mindspore.nn import Adam
from mindsponge import Sponge
from mindsponge import Molecule
from mindsponge import ForceField
from mindsponge.callback import WriteH5MD, RunInfo

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [3]:
system = Molecule(template='water.spce.yaml')

## 分子系统的复制与扩展

在分子对象的最基础的类Molecule中，支持了reduplicate和copy两种复制的模式，其实是同样的操作，不同点在于，reduplicate复制之后会默认把新生成的分子添加到此前定义的分子体系中去。而copy则是创建了一个新的对象，需要通过Molecule内置的append方法，才能把copy出来的对象添加到原始的分子系统中去。

In [4]:
system.reduplicate([0.3, 0, 0])
system.reduplicate([0, 0.3, 0])
new_sys = system.copy([0, 0, -0.3])
system.reduplicate([0, 0, 0.3])
system.append(new_sys)

Molecule<>

In [5]:
potential = ForceField(system, parameters='SPCE')

## 优化器与积分器

在MindSponge与传统深度学习框架的架构图示意中，我们可以发现分子动力学模拟过程中的积分器与深度学习中的优化器，本质上其实是一回事，都需要获取到gradient然后再去对系统进行更新。而如果在分子动力学模拟的过程中直接使用传统优化器如GradientDescent和Adam等来进行能量最小化，也是一个很重要的过程。在执行动力学模拟过程之前，如果不对系统进行能量最小化，将很有可能在运行过程中出现梯度爆炸或者梯度消失的问题，会为系统的正常运行带来众多的不确定性。

![](../../docs/img.png)

In [6]:
opt = Adam(system.trainable_params(), 1e-3)

In [7]:
md = Sponge(system, potential, opt)

TotalTime = 0.39656, [19]
[parse]: 0.0138911
[symbol_resolve]: 0.0741477, [1]
    [Cycle 1]: 0.0734549, [1]
        [resolve]: 0.0734418
[combine_like_graphs]: 2.73809e-06
[inference_opt_prepare]: 0.000630761
[abstract_specialize]: 0.270394
[auto_monad]: 0.000864683
[inline]: 6.96629e-06
[py_pre_ad]: 1.02445e-06
[pynative_shard]: 2.6077e-06
[pipeline_split]: 5.0962e-06
[optimize]: 0.0263499, [15]
    [simplify_data_structures]: 0.00039012
    [opt_a]: 0.0248991, [2]
        [Cycle 1]: 0.023708, [25]
            [expand_dump_flag]: 1.35843e-05
            [switch_simplify]: 0.000803325
            [a_1]: 0.00819018
            [recompute_prepare]: 2.23555e-05
            [updatestate_depend_eliminate]: 2.45553e-05
            [updatestate_assign_eliminate]: 2.1968e-05
            [updatestate_loads_eliminate]: 2.17687e-05
            [parameter_eliminate]: 3.2261e-06
            [a_2]: 0.000862809
            [accelerated_algorithm]: 1.71494e-05
            [auto_parallel]: 4.52809e-06


In [8]:
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, 'tutorial_b03.h5md', save_freq=10)

In [9]:
md.run(1000, callbacks=[run_info, cb_h5md])

TotalTime = 0.0132722, [19]
[parse]: 0.00458984
[symbol_resolve]: 0.00105826, [1]
    [Cycle 1]: 0.00102912, [1]
        [resolve]: 0.00102337
[combine_like_graphs]: 6.03497e-07
[inference_opt_prepare]: 2.8085e-05
[abstract_specialize]: 0.00060638
[auto_monad]: 9.79304e-05
[inline]: 1.62981e-06
[py_pre_ad]: 1.49012e-06
[pynative_shard]: 1.03563e-06
[pipeline_split]: 2.31899e-06
[optimize]: 0.00180736, [15]
    [simplify_data_structures]: 7.74488e-06
    [opt_a]: 0.00146601, [2]
        [Cycle 1]: 0.000863027, [25]
            [expand_dump_flag]: 1.3262e-06
            [switch_simplify]: 9.74163e-06
            [a_1]: 0.000156663
            [recompute_prepare]: 3.61912e-06
            [updatestate_depend_eliminate]: 1.64527e-05
            [updatestate_assign_eliminate]: 3.73647e-06
            [updatestate_loads_eliminate]: 4.15184e-06
            [parameter_eliminate]: 1.6354e-06
            [a_2]: 7.75065e-05
            [accelerated_algorithm]: 6.13555e-06
            [auto_paralle

![](docs/tutorial_b03.gif)